## Old

### Importieren der Klassen

In [ ]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import accelerate

### Festlegen des Modells

In [ ]:
model_id="/home/thsch026/masterarbeit/models/generated/llm-awq/Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"

### Initialisieren des Modells

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=quantization_config, #load_in_8bit=True,#
    device_map='auto',
    token='<EOL>'
)
#peft_model = "BojanaBas/Meta-Llama-3-8B-Instruct-pqa-10"
#model = PeftModel.from_pretrained(base_model, peft_model)

### Inferenz Test

### Autoawq library is needed to test inference with AWQ Models

In [ ]:
pip install autoawq

In [ ]:
# Eingabetext
input_text = "Once upon a time"
#input_text = "Why should I eat healthy?"

# Tokenisiere den Eingabetext
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Messe die Zeit vor der Inferenz
start_time = time.time()

# Führe die Inferenz durch
outputs = model.generate(input_ids, max_length=50)

# Messe die Zeit nach der Inferenz
end_time = time.time()

# Berechne die Inferenz-Zeit
inference_time = end_time - start_time

# Dekodiere die Ausgabe
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
number_of_words = len(output_text.split())
number_of_tokens = len(outputs[0])

token_per_second = number_of_tokens/inference_time

print(f"Inferenz-Zeit: {inference_time} Sekunden")
print(f"Output: {output_text}")
print("Tokens per second: ", token_per_second)



In [ ]:
import csv

# Die neuen Daten, die hinzugefügt werden sollen
neue_daten = ['Lukas', 28]

# Öffne die Datei im Anhangsmodus ('a' steht für append)
with open('daten.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    
    # Schreibe die neuen Daten in die CSV-Datei
    writer.writerow(neue_daten)

print("Neue Daten erfolgreich hinzugefügt.")

In [ ]:
pip install sentencepiece

## Evaluierung Modelle

### Sentencepiece muss ggf. noch mal installiert werden für Mistral (fast tokenizer)

In [ ]:
pip install sentencepiece

In [ ]:
!pip3 install transformers optimum
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

### Laden der libaries und festlegen der GPU

In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
!echo $CUDA_VISIBLE_DEVICES
# torch.cuda.set_device(1)

3


In [2]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig 
import bitsandbytes as bnb
import accelerate
import csv
import packaging
from awq import AutoAWQForCausalLM

### Ergebnis Dateien

In [3]:
full_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference_full.csv"
data_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference_data_only.csv"

##  1. Referenz Modelle

### Llama-3-8B-Instruct

In [ ]:
model_id ="Meta-Llama-3-8B-Instruct"
model_loc = "meta-llama/Meta-Llama-3-8B-Instruct"

### Mistral-7B-Instruct-v0.2

In [ ]:
model_id = "Mistral-7B-Instruct-v0.2"
model_loc = "mistralai/Mistral-7B-Instruct-v0.2"

### Llama-2-7b-chat-hf

In [ ]:
model_id = "Llama-2-7b-chat-hf"
model_loc = "meta-llama/Llama-2-7b-chat-hf"

## 2. AWQ Modelle

### Llama-3-8B-Instruct-HF-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"
model_loc = model_path + model_id

### Mistral-7B-Instruct-v0.2-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Mistral-7B-Instruct-v0.2-AWQ-4bit"
model_loc = model_path + model_id

### Llama-2-7b-chat-HF-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Llama-2-7b-chat-HF-AWQ-4bit"
model_loc = model_path + model_id

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/awq/quant/"
model_id = "Meta-Llama-3-8B-Instruct-HF-w8-g128-awq-v2.pt"
model_loc = model_path + model_id

## 3. Pruned Models (shortened-LLM)

### Meta-Llama-3-8B

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Meta-Llama-3-8B"
model_id = "Meta-Llama-3-8B_pruned_llm-shortened"

### Llama-2-7b-hf (schlechtes Ergebnis)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Llama-2-7b-hf"
model_id = "Llama-2-7b-hf_pruned_llm-shortened"

### Mistral-7B-Instruct

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Mistral-7B-Instruct"
model_id = "Mistral-7B-Instruct_pruned_llm-shortened"

## 4. Lora Models

#### Meta-Llama-3-8B_ms-marco (Kein gutes Ergebnis)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/Meta-Llama-3-8B_ms-marco/model"
model_id = "Meta-Llama-3-8B_ms-marco"

##  5. Qlora Models

### Meta-Llama-3-8B-Instruct_qlora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Meta-Llama-3-8B-Instruct_qlora"
model_id = "Meta-Llama-3-8B-Instruct_qlora"

### Mistral-7B-Instruct-v0.2_qlora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Mistral-7B-Instruct-v0.2_qlora"
model_id = "Mistral-7B-Instruct-v0.2_qlora"

### llama-2-7b-chat_qlora (Fehlt noch)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/llama-2-7b-chat_qlora"
model_id = "llama-2-7b-chat_qlora"

## 6. Prune + Lora Models (shortened-LLM)

#### Meta-Llama-3-8B

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Meta-Llama-3-8B_prune_lora"
model_id = "Meta-Llama-3-8B_prune_lora"

#### Llama-2-7b-hf_prune_lora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Llama-2-7b-hf_prune_lora"
model_id = "Llama-2-7b-hf_prune_lora"

#### Mistral-7B-Instruct_prune_lora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Mistral-7B-Instruct_prune_lora"
model_id = "Mistral-7B-Instruct_prune_lora"

## 7. Prune + Lora + AWQ Models

#### Mistral-7B-Instruct_prune_lora_awq

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Mistral-7B-Instruct-v02_prune_lora_awq"
model_id = "Mistral-7B-Instruct_prune_lora_awq"

#### Meta-Llama-3-8B_prune_lora_awq

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Meta-Llama-3-8B_prune_lora_awq"
model_id ="Meta-Llama-3-8B_prune_lora_awq"

#### Llama-2-7b-chat-hf_prune_lora_awq

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Llama-2-7b-chat-hf_prune_lora_awq"
model_id ="Llama-2-7b-chat-hf_prune_lora_awq"

#### Llama-3-8B-Instruct-prune-lora-gptq

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_lora_gptq/Llama_3_8B_Instruct-prune-lora-gptq"
model_id = "Llama-3-8B-Instruct-prune-lora-gptq"

## Inference Task

### Laden des Modells

In [5]:

tokenizer = AutoTokenizer.from_pretrained(model_loc)

quantization_config = BitsAndBytesConfig()
#quantization_config = BitsAndBytesConfig(bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_loc,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    #quantization_config=quantization_config,
    device_map='cuda'
)
   #safetensors=True,
    #load_in_4bit=True,
    # Bei AWQ Modellen keine quantization config angeben

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
CUDA extension not installed.
CUDA extension not installed.
/home/thsch026/my-envs/inference/lib/python3.10/site-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
Some weights of the model checkpoint at /home/thsch026/masterarbeit/models/generated/prune_lora_gptq/Llama_3_8B_Instruct-prune-lora-gptq were not used when initial

### Interferenz mit Ergebnis Speicherung

In [ ]:
# Funktion für das Speichern der Werte

def append_data(result):
    
    with open(full_csv, mode='a', newline='') as file:
        full_writer = csv.writer(file)
        full_writer.writerow(result)
        
    with open(data_csv, mode='a', newline='') as file:
        data_writer = csv.writer(file)
        data_writer.writerow(result[0:4])

                
device = 'cuda'

question_list = ["Once upon a time", "Why should I eat healthy?", "Is there a second life after death?", "What is the most common injury?","Who is Konrad Zuse?"]

for question in question_list:
    prompt = f'''You are an helpful assistant. USER: {question} ASSISTANT:'''
    input_ids = tokenizer.encode(question, return_tensors='pt')
    # Änderung dr Zeile weil es Probleme mit der Ausführung gab (cuda/cpu anomalie)
    input_ids = tokenizer(question, return_tensors="pt").input_ids.to(device)
    input_ids.to('cuda')
    start_time = time.time()

    # Führe die Inferenz durch
    outputs = model.generate(input_ids, max_length=200)

    # Messe die Zeit nach der Inferenz
    end_time = time.time()

    # Berechne die Inferenz-Zeit
    inference_time = end_time - start_time

    # Dekodiere die Ausgabe
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Berchnung des Ergebnis
    #number_of_words = len(output_text.split())
    number_of_tokens = len(outputs[0])
    memory_usage = round((torch.cuda.memory_allocated() / (1024 **3)),2)
    token_per_second = number_of_tokens/inference_time
    result = [model_id,token_per_second, inference_time, memory_usage, question, output_text]
    print(result)
    append_data(result)
              
    
                                                                
    

    
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Llama-3-8B-Instruct-prune-lora-gptq', 3.146203466066055, 63.56867957115173, 9.51, 'Once upon a time', 'Once upon a time, in a land far, far away, there was a little girl named Alice. She lived in a small village with her parents and her brother, Jack. Alice was a very curious girl, always asking questions and wanting to learn new things.\n\nOne day, while walking through the forest with her brother, Alice came across a strange-looking rabbit. The rabbit was wearing a watch and seemed to be in a hurry.\n\n“Oh no!” exclaimed the rabbit. “I’m going to be late for the tea party!”\n\nAlice was confused. “But it’s only noon,” she said.\n\n“That’s what I said,” replied the rabbit. “But the tea party starts at 4 o’clock, and I’m already an hour late!”\n\nAlice had never heard of a tea party that started at 4 o’clock in the afternoon. She decided to follow']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Llama-3-8B-Instruct-prune-lora-gptq', 4.782308919530669, 41.82080316543579, 9.51, 'Why should I eat healthy?', 'Why should I eat healthy?\n\nEating healthy is important for your overall health and well-being. A healthy diet can help you maintain a healthy weight, reduce your risk of chronic diseases, and improve your mood.\n\nWhat are some tips for eating healthy?\n\n1. Eat plenty of fruits and vegetables.\n2. Choose whole grains over processed grains.\n3. Include lean protein sources in your diet.\n4. Limit your intake of saturated and trans fats.\n5. Eat healthy fats, such as those found in nuts and avocados.\n6. Drink water instead of sugary beverages.\n7. Limit your intake of added sugars and salt.\n8. Get enough calcium and vitamin D.\n9. Eat foods rich in antioxidants.\n10. Avoid processed foods.\n\nWhat are some common myths']


### Beispiel von HF für AWQ Models

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = model_loc

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=True, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

prompt = "Tell me about AI"
prompt_template=f'''{prompt}

'''

print("\n\n*** Generate:")

tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/em_german_mistral_v01-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Warum soll ich gesund essen?"
prompt_template=f'''Du bist ein hilfreicher Assistent. USER: {prompt} ASSISTANT:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))


In [7]:
del model # Specify variable
torch.cuda.empty_cache()